In [2]:
import cobra
from cobra.io import load_model
from pathlib import Path
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
import logging

In [3]:
%cd C:\Users\duart\OneDrive\Documentos\UNIANDES\PHASEOLUS LUNATUS\MODELS\RAVEN\MetaCyc
MetacycPvul_model = read_sbml_model(str("MetacycPvul_model.xml"))

C:\Users\duart\OneDrive\Documentos\UNIANDES\PHASEOLUS LUNATUS\MODELS\RAVEN\MetaCyc


CobraSBMLError: Something went wrong reading the SBML model. Most likely the SBML model is not valid. Please check that your model is valid using the `cobra.io.sbml.validate_sbml_model` function or via the online validator at http://sbml.org/validator .
	`(model, errors) = validate_sbml_model(filename)`
If the model is valid and cannot be read please open an issue at https://github.com/opencobra/cobrapy/issues .

In [3]:
from typing import TYPE_CHECKING, NamedTuple, Optional, Union

import numpy as np
import pandas as pd


# Used to avoid cyclic reference and enable third-party static type checkers to work
if TYPE_CHECKING:
    from cobra import Model


try:
    from scipy.sparse import dok_matrix, lil_matrix
except ImportError:
    dok_matrix, lil_matrix = None, None
    
import pandas
def create_stoichiometric_matrix(
    model: "Model", array_type: str = "dense", dtype: Optional[np.dtype] = None
) -> Union[np.ndarray, dok_matrix, lil_matrix, pd.DataFrame]:
    """Return a stoichiometric array representation of the given model.

    The the columns represent the reactions and rows represent
    metabolites. S[i,j] therefore contains the quantity of metabolite `i`
    produced (negative for consumed) by reaction `j`.

    Parameters
    ----------
    model : cobra.Model
        The cobra model to construct the matrix for.
    array_type : {"dense", "dok", "lil", "DataFrame"}
        The type of array to construct. "dense" will return a standard
        numpy.ndarray. "dok", or "lil" will construct a sparse array using
        scipy of the corresponding type. "DataFrame" will give a
        pandas.DataFrame with metabolite as indices and reaction as
        columns.
    dtype : numpy.dtype, optional
        The desired numpy data type for the array (default numpy.float64).

    Returns
    -------
    matrix of class `dtype`
        The stoichiometric matrix for the given model.

    Raises
    ------
    ValueError
        If sparse matrix is used and scipy is not installed.

    .. deprecated:: 0.18.1
              "DataFrame" option for `array_type` will be replaced with
              "frame" in future versions.

    """
    if array_type not in ("DataFrame", "dense") and not dok_matrix:
        raise ValueError("Sparse matrices require scipy.")

    if dtype is None:
        dtype = np.float64

    array_constructor = {
        "dense": np.zeros,
        "dok": dok_matrix,
        "lil": lil_matrix,
        "DataFrame": np.zeros,
    }

    n_metabolites = len(model.metabolites)
    n_reactions = len(model.reactions)
    array = array_constructor[array_type]((n_metabolites, n_reactions), dtype=dtype)

    m_ind = model.metabolites.index
    r_ind = model.reactions.index

    for reaction in model.reactions:
        for metabolite, stoich in reaction.metabolites.items():
            array[m_ind(metabolite), r_ind(reaction)] = stoich

    if array_type == "DataFrame":
        metabolite_ids = [met.id for met in model.metabolites]
        reaction_ids = [rxn.id for rxn in model.reactions]
        return pd.DataFrame(array, index=metabolite_ids, columns=reaction_ids)

    else:
        return array



In [11]:
array=create_stoichiometric_matrix(Cyc)
Matrix=pd.DataFrame(array)
Matrix.to_csv("Raven-Metacyc_Matrix.csv")
